# 📥 Dependencies

In [ ]:
#@title GPyopt
!pip install GPyOpt

In [ ]:
#@title CPLEX

!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/69b1b89a73a7d0931fbfdb355eb147c3 -O cplex_studio1210.linux-x86-64.bin
!wget https://api.wandb.ai/artifactsV2/gcp-us/veri/QXJ0aWZhY3Q6MjU1ODAzMjI=/97133b747b0114a4e3dba77ab26d68d5 -O response.properties

!pip install docplex
!sh cplex_studio1210.linux-x86-64.bin -f response.properties
!python3 /opt/ibm/ILOG/CPLEX_Studio1210/python/setup.py install

In [ ]:
#@title Tensorflow addons

!pip install tensorflow-addons

In [ ]:
#@title EMLlib

%load_ext autoreload
%autoreload 2

!git clone https://github.com/DanieleVeri/emllib.git

# 📑 Definitions

In [ ]:
#@title Base import and seed

import os
import math
import random
import numpy as np
import tensorflow as tf

import sys
if not 'emllib' in sys.path: sys.path.insert(1, 'emllib')

def set_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    tf.compat.v1.set_random_seed(seed)

##Problem

In [ ]:
#@title Problem class

class Problem:

    def __init__(self, name, fun, input_type, input_bounds, constraint_cb=None):
        self.name = name
        self.fun = fun
        self.input_type = input_type
        self.input_bounds = input_bounds
        self.input_shape = len(self.input_bounds)
        self.constraint_cb = constraint_cb

    def get_dataset(self, n_points):
        x = np.random.rand(n_points, self.input_shape)
        for i, b in enumerate(self.input_bounds):
            lb = b[0]
            ub = b[1]
            if self.input_type[i] == "int":
                x[:,i] = np.random.randint(lb, high=ub, size=n_points)
            else:
                x[:,i] *= ub - lb
                x[:,i] += lb
        y = np.zeros((n_points))
        for i in range(n_points):
            y[i] = self.fun(x[i, :])
        return x, y

    def get_grid(self, n_points):
        x_list = []
        for i, b in enumerate(self.input_bounds):
            lb = b[0]
            ub = b[1]
            if self.input_type[i] == "int":
                x_list.append(np.arange(lb, ub, max(1, (ub-lb)//n_points)))
            else:
                x_list.append(np.arange(lb, ub, (ub-lb)/n_points))
        x = np.array(np.meshgrid(*x_list)).reshape(self.input_shape,-1).T
        y = np.zeros((x.shape[0]))
        for i in range(x.shape[0]):
            y[i] = self.fun(x[i, :])
        return x, y

##TFP utils

In [ ]:
#@title Build and plot

from matplotlib import cm
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

def build_probabilistic_regressor(input_shape, depth=4, width=20):
    mdl = tf.keras.Sequential()
    mdl.add(tf.keras.layers.Input(shape=(input_shape,), dtype='float32'))
    for i in range(depth):
        mdl.add(tf.keras.layers.Dense(width, activation='relu'))
    mdl.add(tf.keras.layers.Dense(2, activation='linear'))
    lf = lambda t: tfp.distributions.Normal(loc=t[:, :1], 
                                            scale=tf.keras.backend.exp(t[:, 1:]))
    mdl.add(tfp.layers.DistributionLambda(lf))
    return mdl

def dlambda_likelihood(y_true, dist):
    return -dist.log_prob(y_true)

def plot_prob_predictions(mdl, x, y):
    prob_pred = mdl(np.expand_dims(x, axis=1))
    pred = prob_pred.mean().numpy().ravel()
    std_pred = prob_pred.stddev().numpy().ravel()
    plt.plot(x, y, c="grey")
    plt.plot(x, pred)
    plt.fill_between(x, pred-std_pred, pred+std_pred, 
                    alpha=0.3, color='tab:blue', label='+/- std')

## Bayesian optimization

In [ ]:
#@title GPy opt - LCB

import GPy
from GPyOpt.methods import BayesianOptimization

def bayesian_opt_gpy(problem, iterations, starting_points):
    set_seed()

    x,y = problem.get_dataset(starting_points)
    if problem.input_shape == 1:
        x = x.reshape(-1, 1)

    bds = []
    for i,b in enumerate(problem.input_bounds):
        bds.append({
            'name': 'X'+str(i), 
            'type': 'continuous', 
            'domain': problem.input_bounds[i]})
        
    def wrapper(in_x):
        return problem.fun(in_x.ravel())

    kernel = GPy.kern.Matern52(input_dim=1, variance=1.0, lengthscale=1.0)
    optimizer = BayesianOptimization(f=wrapper, 
                                    domain=bds,
                                    model_type='GP',
                                    kernel=kernel,
                                    acquisition_type ='LCB',
                                    acquisition_jitter = 0.01,
                                    X=x,
                                    Y=y.reshape(-1,1),
                                    exact_feval=False,
                                    normalize_Y=True,
                                    maximize=False)

    optimizer.run_optimization(max_iter=iterations)
    optimizer.plot_acquisition()
    print("Min found:", optimizer.x_opt, optimizer.fx_opt)
    if ENABLE_WANDB:
        wandb.log({
            "min_found": np.min(optimizer.get_evaluations()[1]),
            "n_iterations": np.argmin(optimizer.get_evaluations()[1])+1-starting_points
        })
    return optimizer

## EML utils

In [ ]:
#@title Parse TFP model

from eml.net.reader import keras_reader

def parse_tfp(model):
    in_shape = model.input_shape[1]
    mdl_no_dist = tf.keras.Sequential()
    mdl_no_dist.add(tf.keras.layers.Input(shape=(in_shape,), dtype='float32'))
    for i in range(len(model.layers)-2):
        w = model.layers[i].weights[1].shape[0]
        mdl_no_dist.add(tf.keras.layers.Dense(w, activation='relu'))
    mdl_no_dist.add(tf.keras.layers.Dense(2, activation='linear'))

    mdl_no_dist.set_weights(model.get_weights())

    nn = keras_reader.read_keras_sequential(mdl_no_dist)
    return nn

In [ ]:
#@title Bounds

from eml.net.process import ibr_bounds

def propagate_bound(parsed_model, bounds):
    bounds = np.array(bounds)
    parsed_model.layer(0).update_lb(bounds[:,0])
    parsed_model.layer(0).update_ub(bounds[:,1])
    ibr_bounds(parsed_model)
    return parsed_model

In [ ]:
#@title Encode model

from eml.backend import cplex_backend
import docplex.mp.model as cpx
from eml.net import embed

def embed_model(bkd, cplex, parsed_model, vtype, bounds):
    mean_lb = parsed_model.layer(-1).lb()[0]  # bounds computed with propagate bounds method
    mean_ub = parsed_model.layer(-1).ub()[0]
    std_lb = parsed_model.layer(-1).lb()[1]
    std_ub = parsed_model.layer(-1).ub()[1]

    xvars = []
    for i,b in enumerate(bounds):
        if vtype[i] == "int":
            xvars.append(cplex.integer_var(lb=b[0], ub=b[1], name="x"+str(i)))
        else:
            xvars.append(cplex.continuous_var(lb=b[0], ub=b[1], name="x"+str(i)))

    yvars = [cplex.continuous_var(lb=mean_lb, ub=mean_ub, name="out_mean"), 
            cplex.continuous_var(lb=std_lb, ub=std_ub, name="out_std")]

    embed.encode(bkd, parsed_model, cplex, xvars, yvars, 'nn')
    return xvars, yvars

In [ ]:
#@title PWL helper

from eml.util import encode_pwl
from scipy.stats import norm

def generate_range(lb, ub, nnodes=7):
    if (lb >= -3 and ub <= 3) or (lb < -3 and ub <= -3) or (ub > 3 and lb >= 3):
        return np.linspace(lb, ub, nnodes)
    if lb < -3 and ub <= 3:
        return np.concatenate((
            np.linspace(lb, -3, min(abs(int(lb+3)), 3), endpoint=False),
            np.linspace(-3, ub, nnodes)
        ))
    if lb >= -3 and ub > 3:
        return np.concatenate((
            np.linspace(lb, 3, nnodes, endpoint=False),
            np.linspace(3, ub, min(abs(int(ub-3)), 3))
        ))
    if lb < -3 and ub > 3:
        return np.concatenate((
            np.linspace(lb, -3, min(abs(int(lb+3)), 3), endpoint=False),
            np.linspace(-3, 3, nnodes, endpoint=False),
            np.linspace(3, ub, min(abs(int(ub-3)), 3))
        ))

def pwl_exp(bkd, cplex, var, nnodes=7):
    xx = generate_range(var.lb, var.ub, nnodes)
    yy = np.array(list(map(math.exp, xx)))
    v = [cplex.continuous_var(lb=0, ub=np.max(yy), name="exp_out"), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_normal_cdf(bkd, cplex, var, nnodes=11):
    xx = generate_range(var.lb, var.ub, nnodes)
    yy = np.array(list(map(norm.cdf, xx)))
    v = [cplex.continuous_var(lb=0, ub=1, name="ncdf_out"), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_normal_pdf(bkd, cplex, var, nnodes=11):
    xx = generate_range(var.lb, var.ub, nnodes)
    yy = np.array(list(map(norm.pdf, xx)))
    v = [cplex.continuous_var(lb=0, ub=1, name="npdf_out"), 
         var]
    encode_pwl(bkd, cplex, v, [yy,xx])
    return v[0]

def pwl_sample_dist(bkd, cplex, vars, samples, vtype, bounds, nnodes=20):
    mapf = lambda x: np.min(np.sum(np.abs(samples - x), axis=1)) / len(bounds)
    p = Problem(None, mapf, vtype, bounds)
    x,y = p.get_grid(nnodes)
    ub = np.diff(np.array(bounds), axis=1)
    ub = np.sum(ub)
    v = [cplex.continuous_var(lb=0, ub=ub, name="dist_out")]+vars
    encode_pwl(bkd, cplex, v, [y,*x.T])
    return v[0]

## 🔄 Optimization loop

In [ ]:
#@title Base Experiment

import time
import tensorflow_addons as tfa

class BaseExperiment:
    def __init__(self, problem, starting_points, iterations, 
                 epochs, lr, weight_decay, depth, width, batch_size, 
                 solver_timeout):
        set_seed()
        self.problem = problem
        self.starting_points = starting_points
        self.iterations = iterations
        self.epochs = epochs
        self.lr = lr
        self.weight_decay = weight_decay
        self.depth = depth
        self.width = width
        self.batch_size = batch_size
        self.solver_timeout = solver_timeout
        self.x_samples, self.y_samples = problem.get_dataset(starting_points)

    def train(self, keras_mdl):
        raise NotImplementedError

    def solver_optimization(self, keras_mdl):
        raise NotImplementedError

    def solution_log(self, solution):
        raise NotImplementedError

    def normalize_X(self, x):
        bounds = np.array(self.problem.input_bounds)
        return (x-bounds[:,0]) / (bounds[:,1]-bounds[:,0])

    def normalize_Y(self, y):
        min, max = np.min(self.y_samples), np.max(self.y_samples)
        return (y-min) / (max-min)

    def reverse_normalize_X(self, norm_x, milp_expr=False):
        bounds = np.array(self.problem.input_bounds)
        if milp_expr:  
            # for cplex variables, np array not supported
            bounds_range = np.squeeze(np.diff(bounds, axis=1))
            xvars = []
            for i,x in enumerate(norm_x):
                xvars.append(x * bounds_range[i] + bounds[i,0])
            return xvars
        return (bounds[:,1]-bounds[:,0])*norm_x + bounds[:,0]

    def reverse_normalize_Y(self, norm_y, stddev=False):
        min, max = np.min(self.y_samples), np.max(self.y_samples)
        if stddev:
            return norm_y*(max-min)
        return (max-min)*norm_y + min

    def plot(self, hstory, keras_mdl):
        if self.problem.input_shape <= 2:
            x,y = self.problem.get_grid(100)
            prob_pred = keras_mdl(self.normalize_X(x))
            pred = prob_pred.mean().numpy().ravel()
            pred = self.reverse_normalize_Y(pred)
            std_pred = prob_pred.stddev().numpy().ravel()
            std_pred = self.reverse_normalize_Y(std_pred, stddev=True)

        plt.plot(hstory.history["loss"])
        plt.savefig('train_loss.png')
        plt.show()
        
        fig = plt.figure(figsize=(15,10))
        if self.problem.input_shape == 1:   # 1D domain
            plt.xlim(self.problem.input_bounds[0])
            x = np.squeeze(x)
            plt.plot(x, y, c="grey")
            plt.plot(x, pred)
            plt.fill_between(x, pred-std_pred, pred+std_pred, 
                            alpha=0.3, color='tab:blue', label='+/- std')
            plt.scatter(self.x_samples, self.y_samples, c="orange")
            plt.legend(["GT", "predicted mean", "predicted CI", "samples"])
            plt.savefig('chart.png')
            plt.show()
        elif self.problem.input_shape == 2:   # 2D domain
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(self.x_samples[:,0], self.x_samples[:,1], self.y_samples, color="orange")
            ax.scatter(x[:,0], x[:,1], y, alpha=0.15, color="lightgrey")
            ax.scatter(x[:,0], x[:,1], pred, alpha=0.3)
            ax.scatter(x[:,0], x[:,1], pred-std_pred, alpha=0.3, color="lightblue")
            ax.scatter(x[:,0], x[:,1], pred+std_pred, alpha=0.3, color="lightblue")
            ax.view_init(elev=15, azim=60)
            plt.legend(["samples", "GT", "predicted mean", "predicted CI"])
            plt.savefig('chart.png')
            plt.show()
        else:
            print("Plot not available for high dimensional domains.")
            print(f"X:\n{self.x_samples}\nY:\n{self.y_samples}")

    def run(self):
        for iteration in range(self.iterations):
            print(f"Iteration {iteration}:", "="*20)
            optimizer = tfa.optimizers.AdamW(weight_decay=self.weight_decay,
                                             learning_rate=self.lr)
            keras_mdl = build_probabilistic_regressor(
                self.problem.input_shape, self.depth, self.width)
            keras_mdl.compile(optimizer=optimizer, loss=dlambda_likelihood)
            keras_mdl.summary()

            train_time = time.time()
            keras_mdl, hstory = self.train(keras_mdl)
            train_time = time.time() - train_time

            self.plot(hstory, keras_mdl)

            solver_time = time.time()
            sol = self.solver_optimization(keras_mdl)
            solver_time = time.time() - solver_time

            if sol is None:
                print(f'Not feasible')
                break
            sol.solve_details.print_information()

            opt_x = np.zeros(self.problem.input_shape)
            for i in range(self.problem.input_shape):
                opt_x[i] = sol["x"+str(i)]
            opt_x = self.reverse_normalize_X(opt_x)
            opt_y = self.problem.fun(opt_x)
            
            emean = self.reverse_normalize_Y(sol['out_mean'])
            print(f"opt input={opt_x}, expected mean={emean}, gt={opt_y}")
            print(f"train time: {train_time}, solver time: {solver_time}")

            self.x_samples = np.concatenate((self.x_samples, np.expand_dims(opt_x, 0)))
            self.y_samples = np.append(self.y_samples, opt_y)

            if iteration == 0 and ENABLE_WANDB: # log starting points before the first iteration
                for j in range(self.starting_points):
                    wandb.log({"x_sample": self.x_samples[j], "y_sample": self.y_samples[j]})
                
            self.solution_log(sol)

            if ENABLE_WANDB: 
                wandb.log({
                    "train_predictions": wandb.Image('chart.png'), 
                    "train_loss": wandb.Image('train_loss.png'),
                    "x_sample": opt_x, "y_sample": opt_y, 
                    "train_time": train_time, "solver_time": solver_time
                })
                
        print(f"Min found: {np.min(self.y_samples)} in {np.argmin(self.y_samples)+1-self.starting_points} iterations")
        if ENABLE_WANDB:
            wandb.log({
                "min_found": np.min(self.y_samples),
                "n_iterations": np.argmin(self.y_samples)+1-self.starting_points
            })

In [ ]:
#@title Base UCB

class BaseUCBExperiment(BaseExperiment):

    def __init__(self, beta_ucb, *args, **kwargs):
        super(BaseUCBExperiment, self).__init__(*args, **kwargs)
        self.beta = beta_ucb

    def solver_optimization(self, keras_mdl):
        print("UCB solver...")
        cplex = cpx.Model()
        bkd = cplex_backend.CplexBackend()

        parsed_mdl = parse_tfp(keras_mdl)
        propagate_bound(parsed_mdl, [[0,1]]*self.problem.input_shape)
        xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                   self.problem.input_type,
                                   [[0,1]]*self.problem.input_shape)

        stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

        ucb_lb = -yvars[0].ub + self.beta * stddev.lb
        ucb_ub = -yvars[0].lb + self.beta * stddev.ub
        print("UCB bounds:", ucb_lb, ucb_ub)
        ucb = cplex.continuous_var(lb=ucb_lb, ub=ucb_ub, name="ucb")
        ucb = -yvars[0] + self.beta * stddev

        # Problem contraints
        if self.problem.constraint_cb is not None:
            rev_norm_xvars = self.reverse_normalize_X(xvars, True)
            self.problem.constraint_cb(self, cplex, rev_norm_xvars)
        
        cplex.set_objective('max', ucb)
        cplex.set_time_limit(self.solver_timeout)
        sol = cplex.solve()

        print(cplex.solve_details)
        cplex.print_information()
        return sol

    def solution_log(self, solution):
        print(f"UCB: {solution['ucb']}")
        if ENABLE_WANDB: 
            wandb.log({
                "ucb": solution['ucb']
            }, commit=False)

In [ ]:
#@title EarlyStop

class EarlyStop(BaseUCBExperiment):

    def __init__(self, patience, eval_points, *args, **kwargs):
        super(EarlyStop, self).__init__(*args, **kwargs)
        self.eval_points = eval_points
        self.x_val, self.y_val = self.problem.get_dataset(self.eval_points)
        self.x_val = self.normalize_X(self.x_val)
        self.y_val = self.normalize_Y(self.y_val)
        self.cb = [tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                    patience=patience, restore_best_weights=True)]
        
    def train(self, keras_mdl):
        print("Early stop training...")
        bs = self.batch_size if self.batch_size else self.x_samples.shape[0]

        norm_x = self.normalize_X(self.x_samples)
        norm_y = self.normalize_Y(self.y_samples)

        hstory = keras_mdl.fit(norm_x, norm_y, 
                               validation_data=(self.x_val, self.y_val),
                               batch_size=bs, epochs=self.epochs, 
                               verbose=0, callbacks=self.cb)

        return keras_mdl, hstory

In [ ]:
#@title Stop CI

class StopCICB(tf.keras.callbacks.Callback):

    def __init__(self, x_val, threshold, *args, **kwargs):
        super(StopCICB, self).__init__(*args, **kwargs)
        self.x_val = x_val
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs={}):
        prob_pred = self.model(self.x_val)
        std_pred = prob_pred.stddev().numpy().ravel()
        if epoch % 100 == 0:
            print(epoch, "mean stddev", np.mean(std_pred))
        if np.mean(std_pred) < self.threshold:
            print("break condition on epoch", epoch, "with mean stddev", np.mean(std_pred))
            self.model.stop_training = True

class StopCI(BaseUCBExperiment):

    def __init__(self, eval_points, ci_threshold, *args, **kwargs):
        super(StopCI, self).__init__(*args, **kwargs)
        self.eval_points = eval_points
        self.ci_threshold = ci_threshold
        val = self.problem.get_dataset(self.eval_points)[0]
        val = self.normalize_X(val)
        stop_ci = StopCICB(val, self.ci_threshold)
        self.cb = [stop_ci]

    def train(self, keras_mdl):
        print("Stop CI training...")
        bs = self.batch_size if self.batch_size else self.x_samples.shape[0]

        norm_x = self.normalize_X(self.x_samples)
        norm_y = self.normalize_Y(self.y_samples)

        hstory = keras_mdl.fit(norm_x, norm_y,
                batch_size=bs, epochs=self.epochs, verbose=0, callbacks=self.cb)

        return keras_mdl, hstory

In [ ]:
#@title AugmentUniform

class AugmentUniform(StopCI):

    def __init__(self, num_aug, *args, **kwargs):
        super(AugmentUniform, self).__init__(*args, **kwargs)
        self.num_aug = num_aug
        self.x_aug_samples = None
        self.y_aug_samples = None

    def plot(self, hstory, keras_mdl):
        if self.problem.input_shape <= 2:
            x,y = self.problem.get_grid(100)
            prob_pred = keras_mdl(self.normalize_X(x))
            pred = prob_pred.mean().numpy().ravel()
            pred = self.reverse_normalize_Y(pred)
            std_pred = prob_pred.stddev().numpy().ravel()
            std_pred = self.reverse_normalize_Y(std_pred, stddev=True)

        plt.plot(hstory.history["loss"])
        plt.savefig('train_loss.png')
        plt.show()

        fig = plt.figure(figsize=(15,10))
        if self.problem.input_shape == 1:   # 1D domain
            plt.xlim(self.problem.input_bounds[0])
            x = np.squeeze(x)
            plt.plot(x, y, c="grey")
            plt.plot(x, pred)
            plt.fill_between(x, pred-std_pred, pred+std_pred, 
                            alpha=0.3, color='tab:blue', label='+/- std')
            plt.scatter(self.x_aug_samples, self.y_aug_samples, c="grey")
            plt.scatter(self.x_samples, self.y_samples, c="orange")
            plt.legend(["GT", "predicted mean", "predicted CI", "samples", "augmented samples"])
            plt.savefig('chart.png')
            plt.show()
        elif self.problem.input_shape == 2:    # 2D domain
            ax = fig.add_subplot(111, projection='3d')
            ax.scatter(x[:,0], x[:,1], y, alpha=0.15, color="lightgrey")
            ax.scatter(self.x_aug_samples[:,0], self.x_aug_samples[:,1], self.y_aug_samples, c="grey", alpha=0.3)
            ax.scatter(self.x_samples[:,0], self.x_samples[:,1], self.y_samples, color="orange", alpha=1)
            ax.scatter(x[:,0], x[:,1], pred)
            ax.scatter(x[:,0], x[:,1], pred-std_pred, alpha=0.3, color="lightblue")
            ax.scatter(x[:,0], x[:,1], pred+std_pred, alpha=0.3, color="lightblue")
            ax.view_init(elev=15, azim=60)
            plt.legend(["samples", "augmented samples", "predicted mean", "predicted CI"])
            plt.savefig('chart.png')
            plt.show()
        else:
            print("Plot not available for high dimensional domains.")
            print(f"X:\n{self.x_samples}\nY:\n{self.y_samples}")

    def train(self, keras_mdl):
        print("Uniform augmented training...")
        self.x_aug_samples = np.concatenate((self.x_samples, 
                                        self.problem.get_dataset(self.num_aug)[0]))
        y_range = np.max(np.abs(self.y_samples))*2                              
        self.y_aug_samples = np.concatenate((self.y_samples, 
                                        (np.random.rand(self.num_aug)-0.5)*y_range))  

        sw = np.ones_like(self.x_aug_samples)  
        sw[:self.x_samples.shape[0]] = self.num_aug*2

        bs = self.batch_size if self.batch_size else self.x_aug_samples.shape[0]

        norm_x = self.normalize_X(self.x_aug_samples)
        norm_y = self.normalize_Y(self.y_aug_samples)

        hstory = keras_mdl.fit(norm_x, norm_y,
            batch_size=bs, epochs=self.epochs, verbose=0, callbacks=self.cb, sample_weight=sw)
        
        return keras_mdl, hstory

In [ ]:
#@title LHS

class LHS(AugmentUniform):

    def train(self, keras_mdl):
        print("LHS augmented training")
        n_datapoint = self.num_aug ** self.problem.input_shape

        aug_x = self.problem.get_grid(self.num_aug)[0]
        aug_x = np.concatenate((aug_x, aug_x))

        y_range = np.max(np.abs(self.y_samples))*2                              
        aug_y = np.concatenate(((np.random.rand(n_datapoint)-0.5)*y_range,
                                (np.random.rand(n_datapoint)-0.5)*y_range))
        self.x_aug_samples = np.concatenate((self.x_samples, aug_x))
        self.y_aug_samples = np.concatenate((self.y_samples, aug_y))

        sw = np.ones_like(self.x_aug_samples)  
        sw[:self.x_samples.shape[0]] = n_datapoint

        bs = self.batch_size if self.batch_size else self.x_aug_samples.shape[0]

        norm_x = self.normalize_X(self.x_aug_samples)
        norm_y = self.normalize_Y(self.y_aug_samples)

        hstory = keras_mdl.fit(norm_x, norm_y,
            batch_size=bs, epochs=self.epochs, verbose=0, callbacks=self.cb, sample_weight=sw)
        
        return keras_mdl, hstory

In [ ]:
#@title MILP dist

class PWLMILPDist(StopCI):

    def solver_optimization(self, keras_mdl):
        print("Distance based UCB solver...")
        cplex = cpx.Model()
        bkd = cplex_backend.CplexBackend()

        parsed_mdl = parse_tfp(keras_mdl)
        propagate_bound(parsed_mdl, [[0,1]]*self.problem.input_shape)
        xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                   self.problem.input_type,
                                   [[0,1]]*self.problem.input_shape)

        stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

        norm_x = self.normalize_X(self.x_samples)
        dist = pwl_sample_dist(bkd, cplex, xvars, norm_x, 
                               self.problem.input_type,
                               [[0,1]]*self.problem.input_shape, 
                               nnodes = min(20, self.x_samples.shape[0]))

        ucb_lb = -yvars[0].ub + self.beta * (stddev.lb + dist.lb)
        ucb_ub = -yvars[0].lb + self.beta * (stddev.ub + dist.ub)
        ucb = cplex.continuous_var(lb=ucb_lb, ub=ucb_ub, name="ucb")
        ucb = -yvars[0] + self.beta * (stddev + dist)
        #ucb = cplex.continuous_var(lb=-cplex.infinity, ub=cplex.infinity, name="ucb")
        #cplex.add_constraint(ucb == -yvars[0] + self.beta * (stddev + dist))

        # Problem contraints
        if self.problem.constraint_cb is not None:
            rev_norm_xvars = self.reverse_normalize_X(xvars, True)
            self.problem.constraint_cb(self, cplex, rev_norm_xvars)

        cplex.set_objective('max', ucb)
        cplex.set_time_limit(self.solver_timeout)
        sol = cplex.solve()
        
        #print("UCB bounds:", ucb_lb, ucb_ub)
        #print("beta", self.beta)
        #print("stdout", sol['exp_out'])
        #print("mean", sol['out_mean'])
        #print("distout", sol['dist_out'])
        #print(norm_x)

        print(cplex.solve_details)
        cplex.print_information()
        return sol

In [ ]:
#@title Hybrid

class Hybrid(PWLMILPDist, AugmentUniform):
    pass

In [ ]:
#@title FastMILP dist

class FastMILPDist(StopCI):

    def solver_optimization(self, keras_mdl):
        print("Distance based UCB solver...")
        cplex = cpx.Model()
        bkd = cplex_backend.CplexBackend()

        parsed_mdl = parse_tfp(keras_mdl)
        propagate_bound(parsed_mdl, [[0,1]]*self.problem.input_shape)
        xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                   self.problem.input_type,
                                   [[0,1]]*self.problem.input_shape)

        stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

        # Distance
        norm_x = self.normalize_X(self.x_samples)
        dist_ub = self.problem.input_shape # hack: max dist = sum(bounds) 
        dist = cplex.continuous_var(lb=0, ub=dist_ub, name="dist")
        for row in range(norm_x.shape[0]):
            sd = 0
            for feature in range(norm_x.shape[1]):
                #sd += (norm_x[row, feature] - xvars[feature]) * (norm_x[row, feature] - xvars[feature]) # NOTE: non-convex
                sd += cplex.abs(norm_x[row, feature] - xvars[feature])
            cplex.add_constraint(sd*self.problem.input_shape >= dist)  # scale down the l1 dist with dimensions

        # UCB
        ucb_lb = -yvars[0].ub + self.beta * (stddev.lb + dist.lb)
        ucb_ub = -yvars[0].lb + self.beta * (stddev.ub + dist.ub)
        ucb = cplex.continuous_var(lb=ucb_lb, ub=ucb_ub, name="ucb")
        ucb = -yvars[0] + self.beta * (stddev + dist)
        #ucb = cplex.continuous_var(lb=-cplex.infinity, ub=cplex.infinity, name="ucb")
        #cplex.add_constraint(ucb == -yvars[0] + self.beta * (stddev + dist))

        # Problem contraints
        if self.problem.constraint_cb is not None:
            rev_norm_xvars = self.reverse_normalize_X(xvars, True)
            self.problem.constraint_cb(self, cplex, rev_norm_xvars)

        cplex.set_objective('max', ucb)
        cplex.set_time_limit(self.solver_timeout)
        sol = cplex.solve()
        
        #cplex.prettyprint()
        #print("UCB bounds:", ucb_lb, ucb_ub)
        print("==== dist", sol['dist'])

        print(cplex.solve_details)
        cplex.print_information()
        return sol

In [ ]:
#@title ☢️ Beta decay

class BetaDecay(StopCI):

    def __init__(self, *args, **kwargs):
        super(BetaDecay, self).__init__(*args, **kwargs)
        self.iter = 1

    def solver_optimization(self, keras_mdl):
        print("Distance based UCB solver...")
        cplex = cpx.Model()
        bkd = cplex_backend.CplexBackend()

        parsed_mdl = parse_tfp(keras_mdl)
        propagate_bound(parsed_mdl, [[0,1]]*self.problem.input_shape)
        xvars, yvars = embed_model(bkd, cplex, parsed_mdl, 
                                   self.problem.input_type,
                                   [[0,1]]*self.problem.input_shape)

        stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=7)

        # Distance
        norm_x = self.normalize_X(self.x_samples)
        dist_ub = self.problem.input_shape #hack
        dist = cplex.continuous_var(lb=0, ub=dist_ub, name="dist")
        for row in range(norm_x.shape[0]):
            sd = 0
            for feature in range(norm_x.shape[1]):
                #sd += (norm_x[row, feature] - xvars[feature]) * (norm_x[row, feature] - xvars[feature])
                sd += cplex.abs(norm_x[row, feature] - xvars[feature])
            cplex.add_constraint(sd*self.problem.input_shape >= dist)  # scale down the l1 dist with dimensions

        # Beta decay
        #current_beta = self.beta * math.pow(self.iter, -1/3)
        current_beta = self.beta * (-self.iter/ITERATIONS + 1 + 1/ITERATIONS)

        # UCB
        ucb_lb = -yvars[0].ub + current_beta * (stddev.lb + dist.lb)
        ucb_ub = -yvars[0].lb + current_beta * (stddev.ub + dist.ub)
        ucb = cplex.continuous_var(lb=ucb_lb, ub=ucb_ub, name="ucb")
        ucb = -yvars[0] + current_beta * (stddev + dist)
        #ucb = cplex.continuous_var(lb=-cplex.infinity, ub=cplex.infinity, name="ucb")
        #cplex.add_constraint(ucb == -yvars[0] + current_beta * (stddev + dist))

        # Problem contraints
        if self.problem.constraint_cb is not None:
            rev_norm_xvars = self.reverse_normalize_X(xvars, True)
            self.problem.constraint_cb(self, cplex, rev_norm_xvars)

        cplex.set_objective('max', ucb)
        cplex.set_time_limit(self.solver_timeout)
        sol = cplex.solve()
        
        #cplex.prettyprint()
        #print("UCB bounds:", ucb_lb, ucb_ub)
        print("==== beta", current_beta)
        print("==== dist", sol['dist'])

        self.iter += 1
        print(cplex.solve_details)
        cplex.print_information()
        return sol

# 🚀 Experiments

In [ ]:
#@title 🎯 Targets

# Functions
################################################################################

def fun_polynomial(x):
    return 0.5*(0.1*math.pow(5*x-1, 4) - 0.4*math.pow(5*x-1, 3) + 0.5*(5*x-1))

def fun_ackley(x):
    x1,x2 = x[0]-1,x[1]-2   # center in 1,2 to remove the 0 bias of nn
    return -20*math.exp(-0.2*math.sqrt(0.5*(x1*x1+x2*x2))) - \
        math.exp(0.5*(math.cos(2*math.pi*x1)+math.cos(2*math.pi*x2))) + \
        math.e + 20

def fun_mccormick(x):
    x1,x2 = x[0],x[1]
    return math.sin(x1+x2) + math.pow(x1-x2,2)-1.5*x1+2.5*x2+1

def build_rosenbrock(rosenbrock_dim):
    def f(x):
        y=0
        for i in range(rosenbrock_dim-1):
            y+= 100 * math.pow(x[i+1] - x[i]*x[i], 2) + math.pow(1-x[i], 2)
        return y
    vtypes = ["real"]*rosenbrock_dim
    bounds = [[-2, 2]]*rosenbrock_dim
    return f, vtypes, bounds

# Constraints 
################################################################################

def cst_disk(experiment, cplex, xvars):     # x1^2 + x2^2 < r^2
    x1,x2 = xvars
    center = [1,2]
    r = 0.5
    x1 -= center[0]
    x2 -= center[1]
    cplex.add_constraint(x1*x1 + x2*x2 <= r*r)

def cst_sphere(experiment, cplex, xvars):     # x1^2 + x2^2 + x3^2 < r^2
    x1,x2,x3 = xvars
    center = [-1,0.5,0.6]
    r = 1.2
    x1 -= center[0]
    x2 -= center[1]
    x3 -= center[2]
    cplex.add_constraint(x1*x1 + x2*x2 + x3*x3 <= r*r)

def cst_leq(experiment, cplex, xvars):     # x1 < x2
    x1,x2 = xvars
    cplex.add_constraint(x1 <= x2)

# Problems
################################################################################

PROBLEM_POLYNOMIAL = Problem("polynomial", 
                             fun_polynomial, 
                             ["real"],
                             [[0, 1]])

PROBLEM_ACKLEY = Problem("ackley", 
                         fun_ackley, 
                         ["real", "real"],
                         [[-3, 3], [-3, 3]])

PROBLEM_CST_ACKLEY = Problem("constrained_ackley", 
                             fun_ackley, 
                             ["real", "real"],
                             [[-3, 3], [-2, 2]], 
                             cst_disk)

PROBLEM_INT_ACKLEY = Problem("int_ackley", 
                             fun_ackley, 
                             ["int", "real"],
                             [[-3, 3], [-2, 2]],
                             cst_leq)

PROBLEM_MCCORMICK = Problem("mccormick", 
                            fun_mccormick, 
                            ["real", "real"],
                            [[-1.5, 4], [-3, 4]])

PROBLEM_ROSENBROCK2D = Problem("rosenbrock2D", *build_rosenbrock(2))
PROBLEM_ROSENBROCK3D = Problem("rosenbrock3D", *build_rosenbrock(3))
PROBLEM_ROSENBROCK4D = Problem("rosenbrock4D", *build_rosenbrock(4))

In [ ]:
#@title 📈 Parameters and WandB

ITERATIONS =                                  10#@param {type:"integer"}
STARTING_POINTS =                              3#@param {type:"integer"}

EPOCHS =                                    9999#@param {type:"integer"}    
PATIENCE = 100                                   #@param {type:"integer"}
EVAL_POINTS = 20                                   #@param {type:"integer"}
LR =                                        1e-4#@param {type:"number"}
WEIGHT_DECAY =                                        1e-4#@param {type:"number"}
BATCH_SIZE = None                               #@param {type:"raw"} None = all samples in 1 batch
DEPTH =                                        4#@param {type:"integer"}
WIDTH =                                       20#@param {type:"integer"}

BETA_UCB =                                     1#@param {type:"number"}

AUG_POINTS =                                 20#@param {type:"integer"}

SOLVER_TIMEOUT = 30                             #@param {type:"integer"}

CI_THRESHOLD =              0.05#@param {type:"number"} 

# set if you plan to log on wandb
ENABLE_WANDB = False                            #@param {type:"boolean"}        

if ENABLE_WANDB and "wandb" not in sys.modules:
    !pip install wandb > /dev/null
    !wandb login
    import wandb

def init_wandb(experiment_name, run_id):
    if run_id is not None: 
        wandb.init(project='eml', id=run_id, resume='allow')
    else:
        wandb.init(project='eml', name=experiment_name)

        wandb.config.iterations = ITERATIONS
        wandb.config.starting_points = STARTING_POINTS
        wandb.config.epochs = EPOCHS
        wandb.config.patience = PATIENCE
        wandb.config.eval_points = EVAL_POINTS
        wandb.config.lr = LR
        wandb.config.weight_decay = WEIGHT_DECAY
        wandb.config.batch_size = BATCH_SIZE
        wandb.config.depth = DEPTH
        wandb.config.width = WIDTH
        wandb.config.beta_ucb = BETA_UCB
        wandb.config.aug_points = AUG_POINTS
        wandb.config.solver_timeout = SOLVER_TIMEOUT
        wandb.config.ci_threshold = CI_THRESHOLD

def run_experiment(name, target):
    if name == "BayesianOptimization":
        optimizer = bayesian_opt_gpy(target, ITERATIONS, STARTING_POINTS)
        return optimizer
        
    if name == "EarlyStop":
        instance = EarlyStop(
            PATIENCE, EVAL_POINTS, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "StopCI":
        instance = StopCI(
            EVAL_POINTS, CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "AugmentUniform":
        instance = AugmentUniform(
            AUG_POINTS, 
            EVAL_POINTS, CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "AugmentLHS":
        instance = LHS(
            AUG_POINTS, 
            EVAL_POINTS, CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "PWLMILPDist":
        instance = PWLMILPDist(
            EVAL_POINTS, CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
        
    if name == "Hybrid":
        instance = Hybrid(
            AUG_POINTS, 
            EVAL_POINTS, CI_THRESHOLD,
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance

    if name == "FastMILPDist":
        instance = FastMILPDist(
            EVAL_POINTS, CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance
    
    if name == "BetaDecay":
        instance = BetaDecay(
            EVAL_POINTS, CI_THRESHOLD, 
            BETA_UCB, 
            target, STARTING_POINTS, ITERATIONS, 
            EPOCHS, LR, WEIGHT_DECAY, DEPTH, WIDTH, BATCH_SIZE, 
            SOLVER_TIMEOUT)
        instance.run()
        return instance

    raise AttributeError("invalid method")

In [ ]:
#@title ▶️ Run experiment

# set if starting a new run
EXPERIMENT_NAME = "constraint_ackley"      #@param {type:"string"}
# set to None if starting a new run
RUN_ID = None                                   #@param {type:"raw"}

if ENABLE_WANDB:
    init_wandb(EXPERIMENT_NAME, RUN_ID)

INSTANCE = run_experiment("Hybrid", PROBLEM_INT_ACKLEY)

if ENABLE_WANDB:
    wandb.finish()

In [ ]:
#@title ⏩ Run all

PROBLEMS = [
    PROBLEM_POLYNOMIAL,
    PROBLEM_ACKLEY,
    PROBLEM_MCCORMICK,
    PROBLEM_ROSENBROCK2D,
    PROBLEM_ROSENBROCK3D,
    PROBLEM_ROSENBROCK4D
]

EXPERIMENT_NAMES = [
#   "BayesianOptimization",
#    "EarlyStop",
#    "StopCI",
#    "AugmentUniform",
#    "AugmentLHS",
#    "PWLMILPDist",
#    "Hybrid",
    "FastMILPDist"
]

for p in PROBLEMS:
    for e in EXPERIMENT_NAMES:
        if ENABLE_WANDB:
            init_wandb(f"{p.name}_{e}", None)
        run_experiment(e, p)

# 🧪 Test

## Problem

In [ ]:
def test_problem():
    def f0(x):
        return 0.5*(0.1*math.pow(5*x-1, 4) - 0.4*math.pow(5*x-1, 3) + 0.5*(5*x-1))
    p0 = Problem(None, f0, [[0, 1]])

    def f1(x):
        x1,x2 = x[0],x[1]
        return 10 * math.sin(x1) * math.sin(x2) 
    p1 = Problem(None, f1, [[-1, 1], [-1, 1]])

    def ackley_fun(x):
        x1,x2 = x[0],x[1]
        return -20*math.exp(-0.2*math.sqrt(0.5*(x1*x1+x2*x2))) - \
            math.exp(0.5*(math.cos(2*math.pi*x1)+math.cos(2*math.pi*x2))) + \
            math.e + 20
    ackley_problem = Problem(None, ackley_fun, [[-3, 3], [-3, 3]])

    print("f0 grid")
    g0 = p0.get_grid(10)
    x0, y0 = g0[0], g0[1]
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(x0, y0)
    plt.show()

    print("f1 grid")
    g1 = p1.get_grid(10)
    x1, y1 = g1[0], g1[1]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x1[:,0], x1[:,1], y1)
    plt.show()

    print("ackley grid")
    g1 = ackley_problem.get_grid(100)
    x1, y1 = g1[0], g1[1]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x1[:,0], x1[:,1], y1)
    plt.show()

    print("f0 dataset")
    d0 = p0.get_dataset(10)
    x0, y0 = d0[0], d0[1]
    fig = plt.figure()
    ax = fig.add_subplot()
    ax.scatter(x0, y0)
    plt.show()

    print("f1 dataset")
    d1 = p1.get_dataset(100)
    x1, y1 = d1[0], d1[1]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x1[:,0], x1[:,1], y1)
    plt.show()

    print("ackley dataset")
    d1 = ackley_problem.get_dataset(10000)
    x1, y1 = d1[0], d1[1]
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.scatter(x1[:,0], x1[:,1], y1)
    plt.show()
    
test_problem()

##TFP

In [ ]:
#@title Whole train set

def tfp_test1():
    set_seed()

    p = Problem(None, polynomial_fun, [[0, 1]])
    x,y = p.get_dataset(100)
    x, y = x.ravel(), y.ravel()
    x_train, y_train = x[:80], y[:80]
    x_val, y_val = x[80:], y[80:]

    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()
    cb = [tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)]
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

    bs = x_train.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_train, y_train, validation_data=(x_val, y_val), 
            batch_size=bs, epochs=5000, verbose=1, callbacks=cb)

    sorted = x_val.argsort()
    plot_prob_predictions(mdl_prob, x_val[sorted], y_val[sorted])

tfp_test1()

In [ ]:
#@title Few points linear

def tfp_test2():
    set_seed()

    x_samples = np.random.rand(5)
    y_samples = x_samples

    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_samples.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_samples, y_samples,
            batch_size=bs, epochs=1000, verbose=1)

    sorted = x_samples.argsort()
    plot_prob_predictions(mdl_prob, x_samples[sorted], y_samples[sorted])

tfp_test2()

In [ ]:
#@title Different variance

def tfp_test3():
    set_seed()

    points = np.random.rand(50)
    noise = np.random.rand(20) / 5
    x_samples = points
    y_samples = np.copy(x_samples)
    sorted_idx = x_samples.argsort()
    y_samples[sorted_idx[:20]] += noise

    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_samples.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_samples, y_samples,
            batch_size=bs, epochs=1000, verbose=1)

    sorted = x_samples.argsort()
    plot_prob_predictions(mdl_prob, x_samples[sorted], y_samples[sorted])

tfp_test3()

## EML

In [ ]:
#@title Minimize cost_fn

def eml_test1():
    #train
    p = Problem(None, polynomial_fun, [[0, 1]])
    x,y = p.get_dataset(100)
    x, y = x.ravel(), y.ravel()
    x_train, y_train = x[:80], y[:80]
    x_val, y_val = x[80:], y[80:]
    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()
    cb = [tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)]
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_train.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_train, y_train, validation_data=(x_val, y_val), 
            batch_size=bs, epochs=5000, verbose=0, callbacks=cb)
    
    #solve
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    parsed_mdl = parse_tfp(mdl_prob)
    propagate_bound(parsed_mdl, [[0,1]])
    xvars, yvars = embed_model(bkd, cplex, parsed_mdl, [[0,1]])

    cplex.set_objective('min', yvars[0])
    cplex.set_time_limit(30)

    sol = cplex.solve()

    print(f'feasible: {sol is not None}')
    print(cplex_backend.model_to_string(cplex))
    print(sol.display())

    opt_x = sol.get_value("x0")
    opt_mean = sol.get_value("out_mean")
    opt_std = math.pow(math.e, sol.get_value("out_std"))
    print(opt_x, opt_mean, opt_std)

    dist = mdl_prob(np.array([[opt_x]]))
    assert dist.mean() - opt_mean < 1e-6
    assert dist.stddev() - opt_std < 1e-6

eml_test1()

In [ ]:
#@title Test pwl

def eml_test2():
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    inp = cplex.continuous_var(lb=0, ub=1, name="test_in")
    out = pwl_normal_pdf(bkd, cplex, inp, nnodes=11)
    cplex.set_objective('max', out)
    cplex.set_time_limit(30)

    cplex.prettyprint()
    sol = cplex.solve()
    print(f'feasible: {sol is not None}')
    print(sol)

    assert sol.get_value("test_in") == 0

eml_test2()

In [ ]:
#@title Minimize EI (non convex error)

#   imp = mu - opt_mean - 0.1
#   Z = imp / var
#   ei = imp * norm.cdf(Z) + var * norm.pdf(Z)

def eml_test3():
    #train
    p = Problem(None, polynomial_fun, [[0, 1]])
    x,y = p.get_dataset(100)
    x, y = x.ravel(), y.ravel()
    x_train, y_train = x[:80], y[:80]
    x_val, y_val = x[80:], y[80:]
    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()
    cb = [tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)]
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_train.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_train, y_train, validation_data=(x_val, y_val), 
            batch_size=bs, epochs=5000, verbose=0, callbacks=cb)
    
    #solve
    epsilon = 0.1
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    parsed_mdl = parse_tfp(mdl_prob)
    propagate_bound(parsed_mdl)
    xvars, yvars = embed_model(bkd, cplex, parsed_mdl)

    # EI
    # find min mean (max improvement)
    cplex.set_objective('min', yvars[0])
    cplex.set_time_limit(30)
    sol = cplex.solve()
    print(f'feasible: {sol is not None}')
    if sol:
        opt_x = sol.get_value("in")
        opt_mean = sol.get_value("out_mean")
        opt_std = math.pow(math.e, sol.get_value("out_std"))
        print(opt_x, opt_mean, opt_std)

    # compute ei
    epsilon = 0.1
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    parsed_mdl = parse_tfp(mdl_prob)
    propagate_bound(parsed_mdl, [[0,1]])
    xvars, yvars = embed_model(bkd, cplex, parsed_mdl, [[0,1]])

    imp = yvars[0] - opt_mean - epsilon
    stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=11)
    Z = cplex.continuous_var(lb=-cplex.infinity, ub=cplex.infinity, name="Z")
    cplex.add_constraint(Z * stddev == imp) # Z = imp / exp(yvars[1])
    ncdf = pwl_normal_cdf(bkd, cplex, Z, nnodes=11)
    npdf = pwl_normal_pdf(bkd, cplex, Z, nnodes=11)
    ei = imp * ncdf + stddev * npdf
    cplex.set_objective('min', ei)

    cplex.set_time_limit(30)
    sol = cplex.solve()
    print(f'feasible: {sol is not None}')
    if sol:
        opt_x = sol.get_value("x0")
    print(sol)

eml_test3()

In [ ]:
#@title Minimize UCB

def eml_test4():
    #train
    p = Problem(None, polynomial_fun, [[0, 1]])
    x,y = p.get_dataset(100)
    x, y = x.ravel(), y.ravel()
    x_train, y_train = x[:80], y[:80]
    x_val, y_val = x[80:], y[80:]
    mdl_prob = build_probabilistic_regressor(1)
    mdl_prob.summary()
    cb = [tf.keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)]
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    bs = x_train.shape[0]
    mdl_prob.compile(optimizer=optimizer, loss=dlambda_likelihood)
    hstory = mdl_prob.fit(x_train, y_train, validation_data=(x_val, y_val), 
            batch_size=bs, epochs=500, verbose=0, callbacks=cb)
    
    #solve
    beta = 0.01
    cplex = cpx.Model()
    bkd = cplex_backend.CplexBackend()

    parsed_mdl = parse_tfp(mdl_prob)
    propagate_bound(parsed_mdl, [[0,1]])
    xvars, yvars = embed_model(bkd, cplex, parsed_mdl, [[0,1]])

    ucb = cplex.continuous_var(lb=-cplex.infinity, ub=cplex.infinity, name="ucb")
    stddev = pwl_exp(bkd, cplex, yvars[1], nnodes=11)
    cplex.add_constraint(ucb == -yvars[0] + beta * stddev) # max -f = min f

    cplex.set_objective('max', ucb)

    cplex.set_time_limit(30)
    sol = cplex.solve()
    print(f'feasible: {sol is not None}')
    if sol:
        opt_x = sol.get_value("x0")
        print(sol)
    return sol, cplex
eml_test4()